In [1]:
import joblib
import pandas as pd
import unicodedata
import re
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [ ]:
csv_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/data finetuned/murs_concat_fine_tuned.csv"
if os.path.exists(csv_path):
	df = pd.read_csv(csv_path)
	print(df.shape)
	print(df.head())
else:
	print(f"❌ File not found: {csv_path}")

❌ File not found: /Users/fabiancordenod/code/fqbq69/BIMpredict-/data_finetuned/murs_concat_fine_tuned.csv


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold

# Supposons que ton dataframe est déjà chargé :
df = df.copy()

# =======================
# 1️⃣ Nettoyage avancé
# =======================

# Remplir les valeurs manquantes basiques
df.fillna(0, inplace=True)

# Supprimer les colonnes très manquantes (ex : plus de 80% de NaN initialement)
missing_ratio = df.isna().mean()
cols_to_drop = missing_ratio[missing_ratio > 0.8].index.tolist()
df.drop(columns=cols_to_drop, inplace=True)
print(f"Colonnes supprimées pour trop de NaN : {cols_to_drop}")


# =======================
# 4️⃣ Réduction de dimension simple
# =======================

# Filtrer uniquement les features numériques pour la variance
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

selector = VarianceThreshold(threshold=0.01)
selector.fit(df[num_cols])

low_variance_cols = num_cols[~selector.get_support()]
print(f"Colonnes à faible variance supprimées : {low_variance_cols.tolist()}")

df.drop(columns=low_variance_cols, inplace=True)

# =======================
# 5️⃣ Résultat final
# =======================

print("✅ Data fine-tuning terminé. Nouvelle forme du dataset :", df.shape)
print(df.head())

# =======================
# 6️⃣ Sauvegarde optionnelle
# =======================

df.to_csv("murs_concat_fine_tuned.csv", index=False)
print("✅ Données sauvegardées sous murs_concat_fine_tuned.csv")

NameError: name 'df' is not defined

In [ ]:
# Définir les targets multi-label
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Nettoyer les noms de colonnes si besoin (reprends ta fonction clean_col)
import unicodedata
import re

def clean_col(col):
    col = unicodedata.normalize('NFKD', str(col)).encode('ascii', 'ignore').decode('utf-8')
    col = col.lower()
    col = re.sub(r'[^a-z0-9]+', '_', col)
    col = col.strip('_')
    return col

df.columns = [clean_col(c) for c in df.columns]

# Vérifier la présence des targets
targets_in_df = [col for col in targets if col in df.columns]
if not targets_in_df:
    raise ValueError(f"Aucune colonne cible trouvée dans le CSV. Colonnes disponibles : {df.columns.tolist()}")

X = df.drop(columns=targets_in_df)
y_multi = df[targets_in_df]

In [ ]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
# Convertir les colonnes catégorielles en string
for col in cat_features:
    X[col] = X[col].astype(str)

# Supprimer les colonnes numériques entièrement vides
num_features = [col for col in num_features if not X[col].isnull().all()]
# Supprimer les doublons dans la liste des colonnes et s'assurer qu'elles existent dans X
all_features = num_features + cat_features
all_features = list(dict.fromkeys(all_features))  # préserve l'ordre et retire les doublons
# S'assurer que chaque colonne n'apparaît qu'une seule fois dans X
all_features = [col for col in all_features if col in X.columns]
X = X.loc[:, ~X.columns.duplicated()]  # retire les colonnes dupliquées dans X
X = X[all_features]

# Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), num_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_features)
    ]
)

# Pipeline complet
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MultiOutputClassifier(RandomForestClassifier(n_estimators=500, random_state=42)))
])

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y_multi, test_size=0.2, random_state=42)

# Supprimer les lignes avec NaN dans les targets
train_notna = y_train.notna().all(axis=1)
test_notna = y_test.notna().all(axis=1)
X_train, y_train = X_train[train_notna], y_train[train_notna]
X_test, y_test = X_test[test_notna], y_test[test_notna]

# Entraînement
pipeline.fit(X_train, y_train)

# Sauvegarde du pipeline
joblib.dump(pipeline, 'randomforestmurspipeline.pkl')
print("✅ Pipeline entraîné et sauvegardé sous randomforestmurspipeline.pkl")

✅ Pipeline entraîné et sauvegardé sous randomforestmurspipeline.pkl
